<a href="https://colab.research.google.com/github/Saim-Hassan786/Learn-Agentic-AI-With-OpenAI-Agents-SDK/blob/main/06-Handoffs/Handoffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handoffs
Handoffs is a very special feature of OpenAI SDK that enhances the power of an Agentic AI Application by enabling it to delegate tasks to other specialized agents with all the conversational or input history and the agent that has the task delgated to will continue from there and perform the operation and can **handoff** to other specialized agent if it thiks it to be necessary.

In [ ]:
# Installing the SDK
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00


In [ ]:
# For running event loop
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Pre requisites SetUp
from google.colab import userdata
GROQ_API_KEY= userdata.get('GROQ_API_KEY')

from agents import set_default_openai_api,set_default_openai_client,set_tracing_disabled
from openai import AsyncOpenAI

external_client = AsyncOpenAI(
    base_url = "https://groq.helicone.ai/openai/v1",
    api_key = GROQ_API_KEY
)
set_default_openai_client(external_client)
set_default_openai_api("chat_completions")
set_tracing_disabled(True)

In [ ]:
# simple runner loop
from agents import Agent,Runner

result = Runner.run_sync(
    starting_agent = Agent(
    name = "Assistant Agent",
    model = "llama3-70b-8192"
),
    input = "what is the capital of sweden"
)
result.final_output

'The capital of Sweden is Stockholm.'

In [ ]:
from agents import Agent,Runner,Handoff,HandoffInputData,function_tool,RunContextWrapper
from pydantic import BaseModel
from agents.extensions import handoff_filters

german_specialist_agent = Agent(
    name = "German Assistant Agent",
    instructions="Translate the users asked test into german",
    model = "llama3-70b-8192",
    handoff_description="German assistant that helps in translation of asked text"
)

@function_tool
def get_weather(city:str):
  return f"The weather in {city} is sunny"

def german_handoff_input_filter(data:HandoffInputData)->HandoffInputData:
#  data = handoff_filters.remove_all_tools(data)
 history = data.input_history[4:]
 print("Handoff Input Filter Occured 🎈 ")
 print("📌 BEFORE Filter:")
 for item in data.input_history:
        print(f"- {item}")

    # Remove all tool calls
 data = handoff_filters.remove_all_tools(data)

 print("📌 AFTER Filter:")
 for item in data.input_history:
        print(f"- {item}")

 return HandoffInputData(
     input_history=(history),
     pre_handoff_items=(data.pre_handoff_items),
     new_items=(data.new_items),
 )

async def on_invoke_handoff_func(ctx: RunContextWrapper, args: str) -> Agent:
    print("Handoff input received from LLM:", args)
    return german_specialist_agent

class HandoffInputSchema(BaseModel):
  user_question : str

german_handoff_agent = Handoff(
    tool_name="german_specialist",
    tool_description="Translate the users asked test into german",
    input_json_schema=HandoffInputSchema.model_json_schema(),
    input_filter=german_handoff_input_filter,
    agent_name=german_specialist_agent.name,
    strict_json_schema=True,
    is_enabled=True,
    on_invoke_handoff= on_invoke_handoff_func
)

main_agent_1 = Agent(
    name = "Main Agent With Tool",
    instructions="Reply with user queries and use tools when required",
    model = "llama3-70b-8192",
    tools = [get_weather]
)

main_agent_2 = Agent(
    name = "Main Agent With Handoff",
    instructions="Reply to user queries **without using any tools**, unless it's a translation request.",
    model = "llama3-70b-8192",
    handoffs=[german_handoff_agent]
)

In [ ]:
result_1 = await Runner.run(
    starting_agent = main_agent_1,
    input = "Hi my name is Saim Hassan"
)
print(result_1.final_output)
print("========"*20)

result_2 = await Runner.run(
    starting_agent = main_agent_1,
    input = result_1.to_input_list() + [{"content":"What is the weather in Berlin","role":"user"}]
)
print(result_2.final_output)
print("========"*20)

result_3 = await Runner.run(
    starting_agent = main_agent_2,
    input = result_2.to_input_list() + [{"content":"What is the green house effect","role":"user"}]
)
print(result_3.final_output)
print("========"*20)

result_4 = await Runner.run(
    starting_agent = main_agent_2,
    input = result_3.to_input_list() + [{"content":"Translate the following text into german 'My name is Saim Hassan, how are you'","role":"user"}]
)
print(result_4.final_output)
print("========"*20)


Hello Saim Hassan! It's nice to meet you. What brings you here today?
The weather in Berlin is sunny.
The greenhouse effect is a natural process that occurs when certain gases in the Earth's atmosphere, such as carbon dioxide, methane, and water vapor, trap heat from the sun, keeping the planet warm enough to support life. This process is essential for life on Earth, but human activities, such as burning fossil fuels and deforestation, have increased the concentration of these gases, leading to an enhancement of the greenhouse effect and an increase in global temperatures, known as global warming.
Handoff input received from LLM: {"user_question":"My name is Saim Hassan, how are you"}
Handoff Input Filter Occured 🎈 
📌 BEFORE Filter:
- {'content': 'Hi my name is Saim Hassan', 'role': 'user'}
- {'id': '__fake_id__', 'content': [{'annotations': [], 'text': "Hello Saim Hassan! It's nice to meet you. What brings you here today?", 'type': 'output_text'}], 'role': 'assistant', 'status': 'comp

In [ ]:
result_4.to_input_list()

[{'content': 'What is the green house effect', 'role': 'user'},
 {'id': '__fake_id__',
  'content': [{'annotations': [],
    'text': "The greenhouse effect is a process that occurs when certain gases in the Earth's atmosphere, such as carbon dioxide, methane, and water vapor, trap heat from the sun, warming the planet. This natural process is essential for life on Earth, as it allows the planet to maintain a habitable temperature. However, human activities, such as burning fossil fuels and deforestation, have increased the concentration of these gases, leading to an enhancement of the greenhouse effect and an increase in global temperatures. This is commonly referred to as global warming.",
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'},
 {'content': "Translate the following text into german 'My name is Saim Hassan, how are you'",
  'role': 'user'},
 {'id': '__fake_id__',
  'content': [{'annotations': [],
    'text': 'Here is the trans

# The deletion of "Introduction" and "Weather" questions and "tool_calls" implies that "handoff_input_filter" worked

# Other Method OF HandOff Creation

In [ ]:
from agents import Agent,Runner,handoff,HandoffInputData,RunContextWrapper
from agents.extensions import handoff_filters
from pydantic import BaseModel

agent_1_spanish = Agent(
    name = "Spanish Translation Agent",
    instructions="Translate the user asked text into spanish",
    model = "llama3-70b-8192",
    handoff_description="Spanish assistant that helps in translation of asked text"
)

class UserInput(BaseModel):
  user_question : str

async def on_handoff_func(ctx:RunContextWrapper,input_data:str)->Agent:
  print("On HandOff Function Called 🎆 ")
  print(input_data.user_question)
  return agent_1_spanish


handoff_1_spanish = handoff(
    agent=agent_1_spanish,
    tool_name_override="spanish_handoff",
    tool_description_override="Spanish assistant that helps in translation of asked text",
    is_enabled=True,
    input_filter=handoff_filters.remove_all_tools,
    on_handoff=on_handoff_func,
    input_type = UserInput
)

main_agent = Agent(
    name = "Main Agent With Handoffs",
    instructions="Reply to user queries **without using any tools**, unless it's a translation request.",
    model = "llama3-70b-8192",
    handoffs=[handoff_1_spanish]
)

In [ ]:
result_with_handoff = await Runner.run(
    starting_agent = main_agent,
    input = "Hi my name is Saim Hassan and I am a boy , translate this into spanish"
)
print(result_with_handoff.final_output)

On HandOff Function Called 🎆 
Hi my name is Saim Hassan and I am a boy
Hola, mi nombre es Saim Hassan y soy un niño.

Here's a breakdown of the translation:

* "Hi" -> "Hola"
* "my name is" -> "mi nombre es"
* "Saim Hassan" remains the same, as it's a proper noun
* "and" -> "y"
* "I am" -> "soy"
* "a boy" -> "un niño"
